# Dacon progect: Prediction Model of Crime Prevention

## Setting

### install

In [1]:
'''pip install pandas
pip install pandas
pip install seaborn
pip install scikit-learn
pip install optuna
pip install lightgbm
pip install xgboost
'''

'pip install pandas\npip install pandas\npip install seaborn\npip install scikit-learn\npip install optuna\npip install lightgbm\npip install xgboost\n'

### import


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import random
import os

import optuna
from optuna import Trial
from optuna.samplers import TPESampler


import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

import xgboost as xgb
from xgboost import XGBClassifier

import sklearn.metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import time

import warnings
warnings.filterwarnings('ignore')

c:\Users\lab\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### seed 고정


In [3]:
#seed 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [4]:
#한글 세팅
fe = fm.FontEntry(fname = 'NanumBarunGothic.ttf', name = 'NanumBarunGothic')

fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='NanumBarunGothic')

### data load

In [5]:
# test data
DATA = pd.read_csv('train.csv')

# test data load
TEST = pd.read_csv('test.csv')

# label data load : 범죄발생구역 중에서 비슷한 범죄항목 비율이 있는 지역끼리 대구분을 해 보았음
#LABEL = pd.read_csv('label.csv')

# day data load : 요일을 월 ~ 일 까지 0~6으로 구분함
DAY = pd.read_csv('day.csv')

#train.head(3)
#label.head(5)

## data EDA

### 범죄발생지역 대구분 - 데이터 머지

In [6]:
DATA.drop(columns=['ID'], inplace=True)
TEST.drop(columns=['ID'], inplace=True)

In [7]:
'''DATA = pd.merge(left=DATA, right=LABEL, left_on='범죄발생지', right_on='구역')
DATA.drop(columns=['구역', 'ID'], inplace=True)

TEST = pd.merge(left=TEST, right=LABEL, left_on='범죄발생지', right_on='구역')
TEST.drop(columns=['구역', 'ID'], inplace=True)
'''

"DATA = pd.merge(left=DATA, right=LABEL, left_on='범죄발생지', right_on='구역')\nDATA.drop(columns=['구역', 'ID'], inplace=True)\n\nTEST = pd.merge(left=TEST, right=LABEL, left_on='범죄발생지', right_on='구역')\nTEST.drop(columns=['구역', 'ID'], inplace=True)\n"

In [8]:
'''
여러개를 기준으로 머지 시킬때는 리스트 형태를 취한다.
마소 엑세스의 테이블끼리 조인해서 쿼리문을 만들때, 화살표와 같은 것!

 data = pd.merge(left=original, right=label, 
                left_on=['data', 'cadavercode'],
                right_on=['date', 'cadavercode']
                )
'''

"\n여러개를 기준으로 머지 시킬때는 리스트 형태를 취한다.\n마소 엑세스의 테이블끼리 조인해서 쿼리문을 만들때, 화살표와 같은 것!\n\n data = pd.merge(left=original, right=label, \n                left_on=['data', 'cadavercode'],\n                right_on=['date', 'cadavercode']\n                )\n"

In [9]:
DATA.head(5)

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET
0,9,화요일,10,137,8.0,2.611124,0.000,0.0,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,2
1,11,화요일,6,438,13.0,3.209093,0.000,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0
2,8,일요일,6,1729,47.0,1.619597,0.000,0.0,0.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,인도,1
3,5,월요일,6,2337,53.0,1.921615,11.375,0.0,0.0,225.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,주거지,1
4,9,일요일,11,1439,41.0,1.789721,0.000,0.0,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주유소,2


In [10]:
TEST.head(5)

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지
0,9,금요일,5,927,28.0,1.570654,19.625000,0.0,0.0,165.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,차도
1,5,수요일,3,926,28.0,1.712457,21.444444,0.0,0.0,175.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,식당
2,5,월요일,6,1437,33.0,0.447496,25.200000,0.0,0.0,290.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지
3,11,화요일,1,1739,31.0,0.878585,0.000000,0.0,0.0,285.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지
4,10,목요일,10,830,15.0,0.496423,26.142857,0.0,0.0,95.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지


### 요일 대구분 - 데이터 머지

In [11]:
DATA = pd.merge(left=DATA, right=DAY, left_on='요일', right_on='요일')
TEST = pd.merge(left=TEST, right=DAY, left_on='요일', right_on='요일')

# 중복 컬럼 데이터 드롭
DATA.drop(columns=['요일'], inplace=True)
TEST.drop(columns=['요일'], inplace=True)


In [12]:
DATA.head(5)

,월,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET,day
0,9,10,137,8.0,2.611124,0.000000,0.0,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,2,1
1,11,6,438,13.0,3.209093,0.000000,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0,1
2,5,8,447,13.0,2.037493,4.125000,0.0,0.0,165.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지,0,1
3,6,12,1438,41.0,1.291232,2.428571,0.0,0.0,105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0,1
4,2,6,1027,41.0,3.015956,0.000000,0.0,336.4,335.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지,0,1


In [13]:
TEST.head(5)

,월,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,day
0,9,5,927,28.0,1.570654,19.625,0.0,0.0,165.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,차도,4
1,5,12,1149,29.0,1.996479,0.000,0.0,0.0,95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,4
2,5,9,137,9.0,4.711117,6.750,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주차장,4
3,10,2,2137,25.0,3.218441,82.000,0.0,0.0,250.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,주거지,4
4,5,1,640,19.0,0.477128,0.000,0.0,0.0,95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지,4


### 범주형 데이터 라벨링

In [14]:
'''cat_feature = ['범죄발생지']

for i in cat_feature:
        
    le = LabelEncoder()

    # 범죄발생지를 LabelEncoder 의 fit_transform 에 넣어준다
    le.fit_transform(DATA[i].unique())

    new_cat = i+"2"
    # 적용!
    DATA[new_cat] = le.transform(DATA[i])
    DATA.drop(columns=i, inplace=True)


for i in cat_feature:
        
    le = LabelEncoder()

    # 범죄발생지를 LabelEncoder 의 fit_transform 에 넣어준다
    le.fit_transform(TEST[i].unique())

    new_cat = i+"2"
    # 적용!
    TEST[new_cat] = le.transform(TEST[i])
    TEST.drop(columns=i, inplace=True)
    '''

'cat_feature = [\'범죄발생지\']\n\nfor i in cat_feature:\n        \n    le = LabelEncoder()\n\n    # 범죄발생지를 LabelEncoder 의 fit_transform 에 넣어준다\n    le.fit_transform(DATA[i].unique())\n\n    new_cat = i+"2"\n    # 적용!\n    DATA[new_cat] = le.transform(DATA[i])\n    DATA.drop(columns=i, inplace=True)\n\n\nfor i in cat_feature:\n        \n    le = LabelEncoder()\n\n    # 범죄발생지를 LabelEncoder 의 fit_transform 에 넣어준다\n    le.fit_transform(TEST[i].unique())\n\n    new_cat = i+"2"\n    # 적용!\n    TEST[new_cat] = le.transform(TEST[i])\n    TEST.drop(columns=i, inplace=True)\n    '

In [15]:
DATA.loc[DATA['범죄발생지'] == '공원', '범죄발생지'] = 0
DATA.loc[DATA['범죄발생지'] == '백화점', '범죄발생지'] = 1
DATA.loc[DATA['범죄발생지'] == '병원', '범죄발생지'] = 2
DATA.loc[DATA['범죄발생지'] == '식당', '범죄발생지'] = 3
DATA.loc[DATA['범죄발생지'] == '약국', '범죄발생지'] = 4
DATA.loc[DATA['범죄발생지'] == '은행', '범죄발생지'] = 5
DATA.loc[DATA['범죄발생지'] == '인도', '범죄발생지'] = 6
DATA.loc[DATA['범죄발생지'] == '주거지', '범죄발생지'] = 7
DATA.loc[DATA['범죄발생지'] == '주유소', '범죄발생지'] = 8
DATA.loc[DATA['범죄발생지'] == '주차장', '범죄발생지'] = 9
DATA.loc[DATA['범죄발생지'] == '차도', '범죄발생지'] = 10
DATA.loc[DATA['범죄발생지'] == '편의점', '범죄발생지'] = 11
DATA.loc[DATA['범죄발생지'] == '학교', '범죄발생지'] = 12
DATA.loc[DATA['범죄발생지'] == '호텔/모텔', '범죄발생지'] = 13

In [16]:
DATA['범죄발생지'] = DATA['범죄발생지'].astype(int)
DATA.dtypes

월            int64
시간           int64
소관경찰서        int64
소관지역       float64
사건발생거리     float64
강수량(mm)    float64
강설량(mm)    float64
적설량(cm)    float64
풍향         float64
안개         float64
짙은안개       float64
번개         float64
진눈깨비       float64
서리         float64
연기/연무      float64
눈날림        float64
범죄발생지        int32
TARGET       int64
day          int64
dtype: object

In [17]:
TEST.loc[TEST['범죄발생지'] == '공원', '범죄발생지'] = 0
TEST.loc[TEST['범죄발생지'] == '백화점', '범죄발생지'] = 1
TEST.loc[TEST['범죄발생지'] == '병원', '범죄발생지'] = 2
TEST.loc[TEST['범죄발생지'] == '식당', '범죄발생지'] = 3
TEST.loc[TEST['범죄발생지'] == '약국', '범죄발생지'] = 4
TEST.loc[TEST['범죄발생지'] == '은행', '범죄발생지'] = 5
TEST.loc[TEST['범죄발생지'] == '인도', '범죄발생지'] = 6
TEST.loc[TEST['범죄발생지'] == '주거지', '범죄발생지'] = 7
TEST.loc[TEST['범죄발생지'] == '주유소', '범죄발생지'] = 8
TEST.loc[TEST['범죄발생지'] == '주차장', '범죄발생지'] = 9
TEST.loc[TEST['범죄발생지'] == '차도', '범죄발생지'] = 10
TEST.loc[TEST['범죄발생지'] == '편의점', '범죄발생지'] = 11
TEST.loc[TEST['범죄발생지'] == '학교', '범죄발생지'] = 12
TEST.loc[TEST['범죄발생지'] == '호텔/모텔', '범죄발생지'] = 13

In [18]:
TEST['범죄발생지'] = TEST['범죄발생지'].astype(int)
TEST.dtypes

월            int64
시간           int64
소관경찰서        int64
소관지역       float64
사건발생거리     float64
강수량(mm)    float64
강설량(mm)    float64
적설량(cm)    float64
풍향         float64
안개         float64
짙은안개       float64
번개         float64
진눈깨비       float64
서리         float64
연기/연무      float64
눈날림        float64
범죄발생지        int32
day          int64
dtype: object

In [19]:
DATA.head(5)

,월,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET,day
0,9,10,137,8.0,2.611124,0.000000,0.0,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,2,1
1,11,6,438,13.0,3.209093,0.000000,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,0,1
2,5,8,447,13.0,2.037493,4.125000,0.0,0.0,165.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,0,1
3,6,12,1438,41.0,1.291232,2.428571,0.0,0.0,105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,0,1
4,2,6,1027,41.0,3.015956,0.000000,0.0,336.4,335.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,0,1


In [20]:

TEST.head(5)

,월,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,day
0,9,5,927,28.0,1.570654,19.625,0.0,0.0,165.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,10,4
1,5,12,1149,29.0,1.996479,0.000,0.0,0.0,95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,4
2,5,9,137,9.0,4.711117,6.750,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,4
3,10,2,2137,25.0,3.218441,82.000,0.0,0.0,250.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,7,4
4,5,1,640,19.0,0.477128,0.000,0.0,0.0,95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,4


In [21]:
# f_data.info()
# test.info()

In [22]:
# 기술통계량을 확인하기위해 describe() 함수를 사용합니다.  
DATA.describe()

,월,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET,day
count,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000
mean,6.430195,6.769507,1060.027581,26.881726,1.912424,24.608776,2.284407,23.430503,186.926107,0.385423,0.017842,0.144042,0.020330,0.010260,0.210755,0.008921,7.944862,0.835355,3.091700
std,3.108302,3.566390,698.380485,13.870968,0.958556,62.711211,15.852881,85.199896,98.299485,0.486698,0.132379,0.351134,0.141128,0.100771,0.407847,0.094030,2.261395,0.819762,2.016831
min,1.000000,1.000000,26.000000,5.000000,0.012269,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,4.000000,526.000000,13.000000,1.209985,0.000000,0.000000,0.000000,95.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,1.000000
50%,7.000000,7.000000,937.000000,27.000000,1.822279,0.625000,0.000000,0.000000,205.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,3.000000
75%,9.000000,10.000000,1638.000000,38.000000,2.476528,18.571429,0.000000,0.000000,260.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,2.000000,5.000000
max,12.000000,12.000000,2450.000000,54.000000,4.998936,614.875000,295.000000,649.800000,360.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,13.000000,2.000000,6.000000


In [23]:
# 범죄발생지의 등장 빈도를 확인하는 코드입니다.  
DATA['범죄발생지'].value_counts()

범죄발생지
7     36077
10    25879
6      6437
11     4835
9      3262
3      1806
1      1493
8      1324
0       736
12      728
4       653
13      591
2       453
5       132
Name: count, dtype: int64

### visualization

In [24]:
'''# 종속변수 분포 확인
sns.displot(DATA['TARGET'])'''

"# 종속변수 분포 확인\nsns.displot(DATA['TARGET'])"

In [25]:
'''# 전체 데이터 분포 확인
except_target = DATA.drop('TARGET', axis = 1)
except_target.hist(figsize = (12,12))
plt.show()'''

"# 전체 데이터 분포 확인\nexcept_target = DATA.drop('TARGET', axis = 1)\nexcept_target.hist(figsize = (12,12))\nplt.show()"

In [26]:
'''# 시간대별 발생 범죄 빈도 확인
sns.countplot(x = '시간', hue = 'TARGET', data = DATA)
plt.show()'''

"# 시간대별 발생 범죄 빈도 확인\nsns.countplot(x = '시간', hue = 'TARGET', data = DATA)\nplt.show()"

In [27]:
'''# 이상치 확인
fig, axes = plt.subplots(2,2, figsize = (10,10))

sns.boxplot(y = DATA['사건발생거리'], ax = axes[0][0])
sns.boxplot(y = DATA['강수량(mm)'], ax = axes[0][1])

sns.boxplot(y = DATA['강설량(mm)'], ax = axes[1][0])
sns.boxplot(y = DATA['적설량(cm)'], ax = axes[1][1])

plt.show()'''

"# 이상치 확인\nfig, axes = plt.subplots(2,2, figsize = (10,10))\n\nsns.boxplot(y = DATA['사건발생거리'], ax = axes[0][0])\nsns.boxplot(y = DATA['강수량(mm)'], ax = axes[0][1])\n\nsns.boxplot(y = DATA['강설량(mm)'], ax = axes[1][0])\nsns.boxplot(y = DATA['적설량(cm)'], ax = axes[1][1])\n\nplt.show()"

In [28]:
'''# 상관관계 확인
plt.figure(figsize = (15,15), dpi = 100)
sns.heatmap(DATA.corr(), annot = True, cmap = 'YlGn')
plt.show()'''

"# 상관관계 확인\nplt.figure(figsize = (15,15), dpi = 100)\nsns.heatmap(DATA.corr(), annot = True, cmap = 'YlGn')\nplt.show()"

## Model: Decision Tree Regression

### 독립변수(x_train), 종속변수(y_train) 분리

In [29]:
# 독립변수(x_train), 종속변수(y_train) 분리
X = DATA.drop('TARGET', axis=1)
y = DATA['TARGET']

x_test = TEST

### 모델 정의

In [30]:
# data  split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [31]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [32]:
def RF_objective(trial):
    """Define the objective function"""

    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        # 'eval_metric': 'mlogloss',
        'use_label_encoder': False
    }

    # Fit the model
    model = xgb.XGBClassifier(**params)
    
    BOOST = model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose = False)
    
    # Make predictions
    y_pred = BOOST.predict(X_valid)
    
    # Evaluate predictions
    F1_SCORE = f1_score(y_valid, y_pred, average="macro")

    return F1_SCORE

In [33]:
RF_study = optuna.create_study(direction='maximize')
RF_study.optimize(RF_objective, n_trials=100)

[I 2023-05-24 17:28:08,133] A new study created in memory with name: no-name-0a8e8319-35e9-4c99-9eb3-1dc3ad6caeea
[I 2023-05-24 17:28:10,344] Trial 0 finished with value: 0.5072246518694559 and parameters: {'max_depth': 6, 'learning_rate': 0.9229400886003543, 'n_estimators': 199, 'min_child_weight': 9, 'gamma': 5.853531261669431e-07, 'subsample': 0.25484993093934394, 'colsample_bytree': 0.01309159157842144, 'reg_alpha': 3.8121306339412613e-07, 'reg_lambda': 1.8308171005297608e-08}. Best is trial 0 with value: 0.5072246518694559.
[I 2023-05-24 17:28:16,674] Trial 1 finished with value: 0.33139241667548497 and parameters: {'max_depth': 3, 'learning_rate': 0.02471305723935588, 'n_estimators': 358, 'min_child_weight': 8, 'gamma': 9.163360510158099e-08, 'subsample': 0.036119488660282986, 'colsample_bytree': 0.07701496403573999, 'reg_alpha': 0.0011048020236533104, 'reg_lambda': 7.783380513497799e-05}. Best is trial 0 with value: 0.5072246518694559.
[I 2023-05-24 17:28:20,075] Trial 2 finishe

In [34]:
trial = RF_study.best_trial

print("  Accuracy: {}".format(trial.value))
print("  Best hyperparameters: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


  Accuracy: 0.5220271611680773
  Best hyperparameters: 
    max_depth: 8
    learning_rate: 0.03621652253135707
    n_estimators: 432
    min_child_weight: 3
    gamma: 0.003651165883147209
    subsample: 0.7727375061622044
    colsample_bytree: 0.6887857758273794
    reg_alpha: 2.497338875659577e-07
    reg_lambda: 0.0009147928104638112


In [35]:
clf = xgb.XGBClassifier(**RF_study.best_params, random_state = 1234, use_label_encoder = False)
clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6887857758273794, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.003651165883147209, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.03621652253135707, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=432, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [36]:
preds = clf.predict(X_valid)
# pred_labels = np.rint(preds)
accuracy = sklearn.metrics.accuracy_score(y_valid, preds)
F1_SCORE = f1_score(y_valid, preds, average="macro")

print("Accuracy: {}".format(accuracy), "F1_score {}".format(F1_SCORE))

Accuracy: 0.5459661177585594 F1_score 0.5186402498584203


### 예측

In [37]:
FINAL_PRED = clf.predict(x_test)

In [38]:
#FINAL_PRED

## 파일 저장

In [39]:
# 파일 저장
submit = pd.read_csv('sample_submission.csv')

# 예측한 값을 TARGET 컬럼에 할당
submit['TARGET'] = FINAL_PRED
submit.head(15)

,ID,TARGET
0,TEST_00000,2
1,TEST_00001,2
2,TEST_00002,1
3,TEST_00003,0
4,TEST_00004,0
5,TEST_00005,2
6,TEST_00006,1
7,TEST_00007,1
8,TEST_00008,0
9,TEST_00009,0


In [40]:
submit.to_csv('submit5_xgboost.csv', index = False)